In [ ]:
!git clone https://github.com/cloner174/mr.Sina.git

In [ ]:
%cd mr.Sina

In [ ]:
# Import Packages :

import pandas as pd
from main import DataHandle


In [ ]:
# Loading data

data = pd.read_csv( 'input/link_dataFinal.csv' , index_col=0)
nodes = pd.read_csv( './input/nodes.csv' )

print(data.shape, nodes.shape)
#((969395, 3), (8977, 5))
data = data.sample(frac=0.01)
nodes_list = []
sors = []
advs = []
pubs = []
for i in data['source'] :
  if i in nodes['id'].tolist():
    if i not in nodes_list:
      sors.append(i)
      nodes_list.append(i)
      index = nodes.index[nodes['id'] == i].tolist()[0]
      if list(nodes.iloc[[index]]['color'] == 'red')[0]:
        advs.append(i)
      else:
        pubs.append(i)
tar = []
for i in data['target'] :
  if i in nodes['id'].tolist():
    if i not in nodes_list:
      tar.append(i)
      nodes_list.append(i)
      index = nodes.index[nodes['id'] == i].tolist()[0]
      if list(nodes.iloc[[index]]['color'] == 'red')[0]:
        advs.append(i)
      else:
        pubs.append(i)

advs_ = list(set(advs))
pubs_ = list(set(pubs))
len(advs_), len(pubs_) #(4299, 1946)
len(nodes_list)# تعداد کل نود ها 6245

In [ ]:
# Finding the nodes from data to pass to the DataHandel class!

data.columns

In [ ]:
data = pd.read_csv( 'input/links.csv' , index_col=0)

data = data.sample(frac=1, random_state=32)

links_ = []
sources_ = []
targets_ = []
for i in range(data.shape[0]) :
    sor_ = data.loc[i, 'source']
    tar_ = data.loc[i, 'target']
    if sor_ in nodes_list and tar_ in nodes_list:
        sources_.append(sor_)
        targets_.append(tar_)
        links_.append( (sor_, tar_) )
        if len(links_) == 60000 :
            break
        else:
            continue

In [ ]:
len(links_) # تعداد کل لینک ها 199981

In [ ]:
"""
nodes_list = []
sors = []
advs = []
pubs = []
for i in data['source'] :
  if i in nodes['id'].tolist():
    if i not in nodes_list:
      sors.append(i)
      nodes_list.append(i)
      index = nodes.index[nodes['id'] == i].tolist()[0]
      if list(nodes.iloc[[index]]['color'] == 'red')[0]:
        advs.append(i)
      else:
        pubs.append(i)
tar = []
for i in data['target'] :
  if i in nodes['id'].tolist():
    if i not in nodes_list:
      tar.append(i)
      nodes_list.append(i)
      index = nodes.index[nodes['id'] == i].tolist()[0]
      if list(nodes.iloc[[index]]['color'] == 'red')[0]:
        advs.append(i)
      else:
        pubs.append(i)
"""

In [ ]:
advs_ = list(set(advs))
pubs_ = list(set(pubs))

#len(advs_), len(pubs_) #(4299, 1946)

In [ ]:
name_laybel1 = [ 'a_node' for i in range( len(advs ))]
name_laybel2 = [ 'p_node' for i in range( len(pubs ))]
name_laybel = name_laybel1.copy()
name_laybel.extend(name_laybel2)
id_laybel = advs.copy()
id_laybel.extend(pubs)
nodes = {
    'name' : name_laybel,
    'id' : id_laybel
}

In [ ]:
links = {
    'source' : sources_,
    'target' : targets_
}

In [ ]:
# getting data ready by turnnig to dicts both !
print(type(nodes), type(links))

In [ ]:
# This will prepare and preprocess the data :

Start = DataHandle( data_links= links, data_nodes= nodes)

In [ ]:
Start.initial_keys()

In [ ]:
layer_one_links, layer_two_links, interconnected_links = Start.modify_links()

In [ ]:
print( 'داده استفاده شده برای ترین وتست - لایه اول لینک ها', len(layer_one_links))
print( 'داده استفاده شده برای ترین وتست - لایه دوم لینک ها', len(layer_two_links))
print( 'داده استفاده شده برای ترین وتست -بین لایه ای لینک ها', len(interconnected_links))



داده ی اولیه !!

داده استفاده شده برای ترین وتست - لایه اول لینک ها 31570

داده استفاده شده برای ترین وتست - لایه دوم لینک ها 137300

داده استفاده شده برای ترین وتست -بین لایه ای لینک ها 31111


In [ ]:
print( 'داده استفاده شده برای ترین وتست - نود ها ادورتایزرز', len(advs))
print( 'داده استفاده شده برای ترین وتست - نود ها پابلیشرز', len(pubs))

In [ ]:
from main import graph

In [ ]:
G = graph.Graph(layer_one_name= 'Advertisers', layer_two_name= 'Publishers', nx_use = True )

In [ ]:
# This is another use of add_links method for simple data :
G_test, G_train , g_nx = G.add_links( layer_one_links = layer_one_links ,
                                               layer_two_links = layer_two_links ,
                                               Interconnected_links = interconnected_links,
                                      layer_one_nodes = advs,
                                      layer_two_nodes = pubs)

In [ ]:
print(' گراف برای تست ', G_test)
print(' گراف برای ترین' , G_train)
print( ' گراف برای ارزیابی نهایی ', g_nx)

In [ ]:
# گراف برای تست  Graph with 3723 nodes and 14987 edges
 #گراف برای ترین Graph with 1740 nodes and 67272 edges
 # گراف برای ارزیابی نهایی  Graph with 5463 nodes and 95553 edges

In [ ]:
import networkx as nx

In [ ]:
# Helper function to create feature set for Logistic Regression
def create_features(graph, edge_list):
    # Feature dictionary
    features = {
        'common_neighbors': [],
        'jaccard_coefficient': [],
        'preferential_attachment': []
    }
    labels = []

    for u, v in edge_list:
        # Common Neighbors
        common_neighbors = list(nx.common_neighbors(graph, u, v))
        features['common_neighbors'].append(len(common_neighbors))

        # Jaccard Coefficient
        jaccard_coeff = list(nx.jaccard_coefficient(graph, [(u, v)]))[0][2]
        features['jaccard_coefficient'].append(jaccard_coeff)

        # Preferential Attachment
        pref_attach = graph.degree(u) * graph.degree(v)
        features['preferential_attachment'].append(pref_attach)

        # Label (1 if edge exists, 0 otherwise)
        labels.append(1 if graph.has_edge(u, v) else 0)

    # Create DataFrame
    feature_df = pd.DataFrame(features)
    return feature_df, labels

# Create features and labels for training and test data
train_features, train_labels = create_features(G_train, list(G_train.edges) + list(nx.non_edges(G_train)))
test_features, test_labels = create_features(G_test, list(G_test.edges) + list(nx.non_edges(G_test)))



print( 'گراف تمرین - تعداد نمونه ها ' ,len(train_labels) )
print('')

print( ' گراف تست - تعداد نمونه ها', len(test_labels) )
print('')
train_features.head()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Initialize and train the logistic regression model
logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(train_features, train_labels)

# Predict on the test set
test_predictions = logistic_model.predict(test_features)
test_probabilities = logistic_model.predict_proba(test_features)[:, 1]

# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)

# Precision
precision = precision_score(test_labels, test_predictions)

# Recall (Sensitivity)
recall = recall_score(test_labels, test_predictions)

# F1 Score
f1 = f1_score(test_labels, test_predictions)

# ROC AUC Score (if y_pred contains probabilities)
roc_auc = roc_auc_score(test_labels, test_predictions)

# Confusion Matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)


In [ ]:
new_test_features, new_test_labels = create_features(g_nx, list(g_nx.edges) + list(nx.non_edges(g_nx)))

In [ ]:
new_test_predictions = logistic_model.predict(new_test_features)

In [ ]:
# Accuracy
accuracy = accuracy_score(new_test_labels, new_test_predictions)

# Precision
precision = precision_score(new_test_labels, new_test_predictions)

# Recall (Sensitivity)
recall = recall_score(new_test_labels, new_test_predictions)

# F1 Score
f1 = f1_score(new_test_labels, new_test_predictions)

# ROC AUC Score (if y_pred contains probabilities)
roc_auc = roc_auc_score(new_test_labels, new_test_predictions)

# Confusion Matrix
conf_matrix = confusion_matrix(new_test_labels, new_test_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Centrality Measures :

def calculate_centrality_measures(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    return degree_centrality, betweenness_centrality, closeness_centrality

In [ ]:
degree_centrality = nx.degree_centrality(G_train)
closeness_centrality = nx.closeness_centrality(G_train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

degree_values = list(degree_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(degree_values, bins = 20, kde=True, color='blue')
plt.title('Degree Centrality Distribution G_train')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()

closeness_values = list(closeness_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(closeness_values, kde=True, color='red')
plt.title('Closeness Centrality Distribution G_train')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()


In [ ]:
degree_centrality = nx.degree_centrality(G_test)
closeness_centrality = nx.closeness_centrality(G_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

degree_values = list(degree_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(degree_values, bins = 20, kde=True, color='blue')
plt.title('Degree Centrality Distribution G_test')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()

closeness_values = list(closeness_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(closeness_values, kde=True, color='red')
plt.title('Closeness Centrality Distribution G_test')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()


In [ ]:
degree_centrality = nx.degree_centrality(g_nx)
closeness_centrality = nx.closeness_centrality(g_nx)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

degree_values = list(degree_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(degree_values, bins = 20, kde=True, color='blue')
plt.title('Degree Centrality Distribution G_between_layers')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()

closeness_values = list(closeness_centrality.values())
plt.figure(figsize=(10, 6))
sns.histplot(closeness_values, kde=True, color='red')
plt.title('Closeness Centrality Distribution G_between_layers')
plt.xlabel('Centrality')
plt.ylabel('Frequency')
plt.show()


ابتدا سری به روش جنگل تصادفی میزنیم چون با استخراج ویژگی های عددی الان یک پیش بینی پیژگی محور محض را داریم\\

باید حلقه ی ویژگی ها ۳ بار تککرار شود


In [ ]:
G = G_train

In [ ]:
G = G_test

In [ ]:
G = g_nx

In [ ]:

from itertools import combinations

# Graph
degree_centrality, betweenness_centrality, closeness_centrality = calculate_centrality_measures(G)
features = []
labels = []

# Calculating features for each pair of nodes:
for node1, node2 in combinations(G.nodes(), 2):

    # Centrality measures for node1 and node2
    degree_centrality_1 = degree_centrality[node1]
    degree_centrality_2 = degree_centrality[node2]
    betweenness_centrality_1 = betweenness_centrality[node1]
    betweenness_centrality_2 = betweenness_centrality[node2]
    closeness_centrality_1 = closeness_centrality[node1]
    closeness_centrality_2 = closeness_centrality[node2]

    # Directly compute and append features
    features.append((
        node1,
        node2,
        len(list(nx.common_neighbors(G, node1, node2))),  # Common Neighbors
        next(nx.preferential_attachment(G, [(node1, node2)]))[2],  # Preferential Attachment
        next(nx.resource_allocation_index(G, [(node1, node2)]))[2],  # Resource Allocation
        next(nx.adamic_adar_index(G, [(node1, node2)]))[2],  # Adamic Adar
        next(nx.jaccard_coefficient(G, [(node1, node2)]))[2] , # Jaccard Coefficient
        (degree_centrality_1 + degree_centrality_2) / 2,
        (betweenness_centrality_1 + betweenness_centrality_2) / 2,
        (closeness_centrality_1 + closeness_centrality_2) / 2
    ))

    # Existence of links between nodes (1 or 0)
    labels.append(1 if G.has_edge(node1, node2) else 0)

# Creating a DataFrame from the features and labels
features_df = pd.DataFrame(features,
                           columns=['node1','node2','Common Neighbors',
                                    'Preferential Attachment', 'Resource Allocation',
                                    'Adamic Adar', 'Jaccard Coefficient', 'Average Degree Centrality',
                                    'Average Betweenness Centrality', 'Average Closeness Centrality'])

features_df['Label'] = labels

# Now features_df is ready to use in a machine learning model


In [ ]:
features_df.to_csv('G-Layer1-dataframe.csv', index=False,
                   compression = {'method': 'zip', 'compresslevel': 1 } )

# This furtehr can be loaded using compression='zip' !

In [ ]:
features_df.to_csv('G-Layer2-dataframe.csv', index=False, 
                   compression = {'method': 'zip', 'compresslevel': 1 } )
# This furtehr can be loaded using compression='zip' !

In [ ]:
features_df.to_csv('G-inBetweenLayers-dataframe3.csv', index=False, 
                   compression = {'method': 'zip', 'compresslevel': 8 } )
# This furtehr can be loaded using compression='zip' !

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
%cd ./mr.Sina

In [ ]:
data1 = pd.read_csv('./G-Layer1-dataframe.csv', compression='zip')
data2 = pd.read_csv('./G-Layer2-dataframe.csv', compression='zip')
data3 = pd.read_csv('./G-inBetweenLayers-dataframe.csv', compression='zip')

In [ ]:
y1 = data1['Label']  # Labels
X1 = data1.drop(['node1','node2','Label'], axis=1)  # Features

In [ ]:
y2 = data2['Label']  # Labels
X2 = data2.drop(['node1','node2','Label'], axis=1)  # Features

In [ ]:
X3 = data3.drop(['node1','node2','Label'], axis=1)  # Features
y3 = data3['Label']  # Labels

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

In [ ]:
#X_train1.shape, X_train2.shape, X_train3.shape, y_train1.shape, y_train2.shape, y_test3.shape

In [ ]:
X_train_ = np.concatenate((X_train1, X_train2, X_train3))
y_train_ = np.concatenate((y_train1, y_train2, y_train3))

X_test_ = np.concatenate((X_test1, X_test2, X_test3))
y_test_ = np.concatenate((y_test1, y_test2, y_test3))

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix


# Prepare data
X = np.array(X_train_)
y = np.array(y_train_)

# model :
clf = RandomForestClassifier(n_estimators=100, random_state=8, verbose = 1)

# Stratified K-Fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=8)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train model
    clf.fit(X_train, y_train)

    # Predict probabilities
    probabilities = clf.predict_proba(X_test)[:, 1]
    predict = clf.predict(X_test)

    predictions = probabilities.astype(int)

    # Calculate metrics
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, predictions, average='binary')
    accuracy = accuracy_score(y_test, predict)
    print(f"Average Accuracy: {accuracy}")
    print(f"Average Precision: {precision}")
    print(f"Average Recall: {recall}")
    print(f"Average F1-Score: {fscore}")

In [ ]:
# unseen Data :

predict_ = clf.predict(X_test_)
predict_prob = clf.predict_proba(X_test_)[:, 1]
predict_prob = predict_prob.astype(int)

accuracy_ = accuracy_score(y_test_, predict_)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test_, predict_prob, average='binary')

print(f"Average Accuracy: {accuracy_}")
print(f"Average Precision: {precision}")
print(f"Average Recall: {recall}")
print(f"Average F1-Score: {fscore}")


In [ ]:
confuse_matrix = []

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_test_, predict_)
plt.title('Confusion Matrix for Model Predictions X_test1')
plt.show()

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_test_, predict_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic y_test1 VS test_probabilities1')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Accuracy
accuracy = accuracy_score(y_test_, predict_)

# Precision
precision = precision_score(y_test_, predict_prob)

# Recall (Sensitivity)
recall = recall_score(y_test_, predict_prob)

# F1 Score
f1 = f1_score(y_test_, predict_prob)

# ROC AUC Score (if y_pred contains probabilities)
roc_auc = roc_auc_score(y_test_, predict_prob)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_, predict_)



In [ ]:
# Creating DataFrame
X_test_results_df = pd.DataFrame({
    'Actual from X_test': y_test_,
    'Predicted from X_test': predict_,


})

X_test_results_df_prob = pd.DataFrame({
    'Actual from X_test': y_test_,
    'Predicted Prob from X_test': predict_prob,
})

# Saving DataFrame to CSV
csv_path = 'X_test_results_df.csv'
X_test_results_df.to_csv(csv_path, index=False)

csv_path = 'X_test_results_df_prob.csv'
X_test_results_df_prob.to_csv(csv_path, index=False)

print("Saved predictions and actual labels")



In [ ]:
# Confusion Matrix Guide :
#    [ [TN    FP]
#      [FN    TP] ]

In [ ]:
#%pip install torch_geometric

بگذارید رگرسیون را تست کنیم

In [ ]:
# data1 = train, data2 = train , data3 = test

In [ ]:
X_train1 = data1.drop(['node1','node2','Label'], axis=1)  # Features
y_train1 = data1['Label']  # Labels

X_train2 = data2.drop(['node1','node2','Label'], axis=1)  # Features
y_train2 = data2['Label']  # Labels

X_test = data3.drop(['node1','node2','Label'], axis=1)  # Features
y_test = data3['Label']  # Labels

In [ ]:
X_train = np.concatenate((X_train1, X_train2, X_train3))
y_train = np.concatenate((y_train1, y_train2, y_train3))

In [ ]:
train_features = X_train
train_labels = y_train

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

# Initialize and train the logistic regression model
logistic_model = LogisticRegression(random_state=64, max_iter=1000)
logistic_model.fit(train_features, train_labels)

In [ ]:
X_test = np.concatenate((X_test1, X_test2, X_test3))
y_test = np.concatenate((y_test1, y_test2, y_test3))

In [ ]:
# Now the completly new and unseen :
test_features = X_test
test_labels = y_test

In [ ]:
# Predict on the test set
test_predictions = logistic_model.predict(test_features)
test_probabilities = logistic_model.predict_proba(test_features)[:, 1]

In [ ]:
# Calculate metrics
roc_auc = roc_auc_score(test_labels, test_probabilities)
accuracy = accuracy_score(test_labels, test_predictions)

roc_auc, accuracy

In [ ]:
# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)

# Precision
precision = precision_score(test_labels, test_predictions)

# Recall (Sensitivity)
recall = recall_score(test_labels, test_predictions)

# F1 Score
f1 = f1_score(test_labels, test_predictions)

# ROC AUC Score (if y_pred contains probabilities)
roc_auc = roc_auc_score(test_labels, test_probabilities)

# Confusion Matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(test_labels, test_probabilities)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic ->LogisticRegression<- ')
plt.legend(loc="lower right")
plt.show()


In [ ]:
data1 = pd.read_csv('./G-Layer1-dataframe.csv', compression='zip')
data2 = pd.read_csv('./G-Layer2-dataframe.csv', compression='zip')
data3 = pd.read_csv('./G-inBetweenLayers-dataframe.csv', compression='zip')
y1 = data1['Label']  # Labels
X1 = data1.drop(['node1','node2','Label'], axis=1)  # Features
y2 = data2['Label']  # Labels
X2 = data2.drop(['node1','node2','Label'], axis=1)  # Features
X3 = data3.drop(['node1','node2','Label'], axis=1)  # Features
y3 = data3['Label']  # Labels

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=16)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=16)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=16)

In [ ]:
X_train_ = np.concatenate((X_train1, X_train2, X_train3))
y_train_ = np.concatenate((y_train1, y_train2, y_train3))

X_test_ = np.concatenate((X_test1, X_test2, X_test3))
y_test_ = np.concatenate((y_test1, y_test2, y_test3))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score

# Define the parameter grid
param_grid = {
    'n_estimators': [20, 30],  # Number of trees in the forest
    'max_depth': [100, 200, 300],  # Maximum depth of the tree
    'min_samples_split': [50, 100]   # Minimum number of samples required to split an internal node
}

# Initialize the classifier
rf = RandomForestClassifier(random_state=32)

# Create the grid search object
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=5)

# Fit grid search to the data
X_train, X_test, y_train, y_test = train_test_split(X_train_, y_train_, test_size=0.2, random_state=32)
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Use the best parameters to make predictions
best_clf = grid_search.best_estimator_
predictions = best_clf.predict(X_test_)
print("Test Accuracy: ", accuracy_score(y_test_, predictions))


In [ ]:
predictions = best_clf.predict(X_test_)
probabilities = best_clf.predict_proba(X_test_)[:, 1]

In [ ]:
# Accuracy
accuracy = accuracy_score(y_test_, test_predictions)

# Precision
precision = precision_score(y_test_, test_predictions)

# Recall (Sensitivity)
recall = recall_score(y_test_, test_predictions)

# F1 Score
f1 = f1_score(y_test_, test_predictions)

# ROC AUC Score (if y_pred contains probabilities)
roc_auc = roc_auc_score(y_test_, test_probabilities)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_, test_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)